## Project Overview

### Dataset:

https://www.kaggle.com/datasets/naserabdullahalam/phishing-email-dataset/data?select=phishing_email.csv

 

### Project Objective:

The objective of the project is to classify emails using supervised learning methods. Emails will be categorized into several classes such as:

- Spam
- Normal emails
- Phishing
- Fraud

The dataset is composed of several separate sources:

    Enron and Ling Datasets: primarily focused on the core content of emails.
    CEAS, Nazario, Nigerian Fraud, and SpamAssassin Datasets: provide context about the message, such as the sender, recipient, date, etc.

The data will require preprocessing to create a unified database that includes all necessary information. The entire project consists of approximately 85,000 emails.

## Preprocessing

Pierwsza częśc preprocessingu zawiera **import** odpowiednich bilbiotek oraz **paczek nltk** (biblioteki do przetwarzania tekstu) oraz **stworzenia** zbioru **stop_words** - słów, które niosą niską zawartość informacyjną.

In [ ]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer

In [ ]:
def prepare_nltk() -> None:
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('omw-1.4')


prepare_nltk()
stop_words_set = set(nltk.corpus.stopwords.words('english'))

### Podział na różne kategorie

**Finalny dataset**, będzie składał się z danych z **6 różnych zbiorów danych**. Każdy z tych dataset-ów zawiera odpowiednie kategorie (label), która oznacza wiadomość **legit (label 0) bądź nie (label 1)**. Jednak różne datasety zawierają różne informacje dotyczące samych wiadomości i **etykiety mają inne znaczenie w zależności od zbioru danych**. Z tego powodu każdy dataset został przypisany do konkretnej kategorii:

| Dataset name | Category | Label of faked messages |
| --- | --- | --- |
| **Enron** | Spam | **1** |
| **Ling** | Spam | **1** |
| **SpamAssasin** | Spam | **1** |
| **CEAS_08** | Phishing | **2** |
| **Nazario** | Phishing | **2** |
| **Nigerian_Fraud** | Fraud | **3** |




In [ ]:
list_of_spam_dataset = [
    "dataset/non-processed/SpamAssasin.csv",
    "dataset/non-processed/Enron.csv",
    "dataset/non-processed/Ling.csv"
]
list_of_fraud_dataset = [
    "dataset/non-processed/Nigerian_Fraud.csv"
]
list_of_phishing_dataset = [
    "dataset/non-processed/CEAS_08.csv",
    "dataset/non-processed/Nazario.csv"
]


final_data_frame = pd.DataFrame()

for dataset_path in list_of_spam_dataset:

    df = manage_dataset_preprocess(dataset_path)
    df['label'] = df['label'].map({0: 0, 1: 1})
    final_data_frame = pd.concat([final_data_frame, df], ignore_index=True)

for dataset_path in list_of_phishing_dataset:
    
    df = manage_dataset_preprocess(dataset_path)
    df['label'] = df['label'].map({0: 0, 1: 2})
    final_data_frame = pd.concat([final_data_frame, df], ignore_index=True)


for dataset_path in list_of_fraud_dataset:
    
    df = manage_dataset_preprocess(dataset_path)
    df['label'] = df['label'].map({0: 0, 1: 3})
    final_data_frame = pd.concat([final_data_frame, df], ignore_index=True)

### Ogólny preprocessing danych

Każdy dataset zawiera różne kolumny. Każdy z nich zawiera kolumnę subject oraz body, jednak niektóre z nich nie zawierają kolumn receiver i sender (Jeśli dataset nie zawiera
jednej z tych kolumn, wartość poszczególnych wartości jest ustawiona na 'None'). Finalna baza danych, będzie zawierała 5 kolumn:

- **body**
- **subject**
- **receiver**
- **sender**
- **label**

Każda z tych kolumn zostanie przetworzona przez osobną funkcję.

In [ ]:
def manage_dataset_preprocess(dataset_path: str) -> pd.DataFrame:
    df = pd.read_csv(dataset_path)

    if 'sender' not in df.columns:
        df['sender'] = None
    if 'receiver' not in df.columns:
        df['receiver'] = None
    
    df = df[['subject', 'body', 'sender', 'receiver',  'label']]
    df['body'] = df['body'].apply(preprocess_dataset_text)
    df['subject'] = df['subject'].apply(preprocess_dataset_subject)
    df['receiver'] = df['receiver'].apply(preprocess_dataset_sender_receiver)
    df['sender'] = df['sender'].apply(preprocess_dataset_sender_receiver)

    return df


### Sposób przetwarzania kolumn

Każdy ciąg znaków z każdej kolumny (oprócz 'label') jest **przetwarzany za pomocą regex**, aby pozbyć zredukować ilość danych i ograniczyć liczbę śmieci. Tabela, z danymi w każdej kolumnie:

| Column | delete E-mail | delete links | only chars | delete garbage | Lower letters | Lematizer |
| --- | --- | --- | --- | --- | --- | --- |
| **Body** | ✅ | ✅ | ✅ | ✅ | ✅ | ✅ |
| **Subject** | ❌ | ❌ | ✅ | ❌ | ✅ | ✅ |
| **Sender**, **Receiver** | ❌ | ❌ | ❌ | ❌ | ✅ | ❌ |

*garbage word* - słowo które zawiera 4 powtórzenia z rzędu tej samej litery, lub jest dłuższe niż 15 znaków. Słowa śmieci zostały usunięte tylko z kolumny 'body', ponieważ to tam jest największa potrzeba ograniczenia ilości danych.

Oprócz tego z kolumn **'body' oraz 'subject' usuwane** są tak zwane **'stop_words'** (O tym w dalszej części). W kolumnach 'sender' oraz 'receiver' dozwolone znaki to: '.', '-' oraz '_'. W tych dwóch kolumnach **obcinamy** także **'username' emaila**, a **pozostawamy** **tylko nazwę domeny** oraz **top level domain**.

Jeśli dane wejściowe **nie są instancją string**, lub przetworzony **ciąg znaków jest pusty** to zwracana jest wartość **'None'**.

In [ ]:
def preprocess_dataset_text(text: str) -> str | None:

    if not isinstance(text, str):
        return None
    
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    ### Remove garbage
    text = re.sub(r'\b([a-zA-Z])\1{4,}\b', '', text)  # Same letter more than 5 times
    text = re.sub(r'\b\w{15,}\b', '', text)  # Words longer than 15 chars

    text = text.lower()
    text = " ".join(word for word in text.split() if word not in stop_words_set)

    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])

    if text == '':
        return None

    return text


def preprocess_dataset_subject(subject: str) -> str | None:
    
    if not isinstance(subject, str) or subject == '':
        return None
    
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    subject = subject.lower()
    subject = re.sub(r'[^a-zA-Z\s]', '', subject)

    # Stop_words
    subject = " ".join(word for word in subject.split() if word not in stop_words_set)
    subject = " ".join([lemmatizer.lemmatize(word) for word in subject.split()])  # 4. Lematyzacja

    if subject == '':
        return None

    return subject


def preprocess_dataset_sender_receiver(data: str) -> str | None:
    
    if not isinstance(data, str):
        return None

    data = data.strip().lower()
    data = data.split('@')[-1]        # TODO: Only domain

    match = re.match(r'^[a-z0-9._-]*', data)

    if match:
        data = match.group(0)

    if data == '':
            return None

    return data

### Rozwiązanie problemu pustych wartości lub wartości 'None'

Niektóre datasety już na starcie nie zawierały pewnych wymaganych danych:

| Dataset | body | subject | sender | receiver |
| --- | --- | --- | --- | --- |
| **Enron** | ✅ | ✅ | ❌ | ❌ |
| **Ling** | ✅ | ✅ | ❌ | ❌ |
| **SpamAssasin** | ✅ | ✅ | ✅ | ✅ |
| **CEAS_08** | ✅ | ✅ | ✅ | ✅ |
| **Nazario** | ✅ | ✅ | ✅ | ✅ |
| **Nigerian_Fraud** | ✅ | ✅ | ✅ | ✅ |

Oprócz tego, część danych została utracona poprzez filtrowanie i przetwarzanie kolumn (wszytskie puste dane zostały zastąpione wartośćią 'None'). Z tego względu wartości 'None' zostaną następnie zastąpione przez najczęściej występującą wartość w każdej z kolumn.

In [ ]:
def impute_missing_values(df: pd.DataFrame, column: str) -> None:
    mode_value = df[column].mode()[0]
    df.fillna({column: mode_value}, inplace=True)


impute_missing_values(final_data_frame, "subject")
impute_missing_values(final_data_frame, "body")
impute_missing_values(final_data_frame, "sender")
impute_missing_values(final_data_frame, "receiver")

### Ostateczne zbiory danych

Przetworzone dane zostaną zapisane do pliku '.csv'. Z racji tego, że projekt skupia się na przetestowaniu rónych modeli, o różnej charakterystyce, ostatecznie stworzone zostały 4 różne datasety:

- **final.csv**
- **final-domain-only.csv**
- **final-with-stop-words.csv**
- **final-with-stop-words-domain-only.csv**

| Final Dataset name | Przekształcenie 'sender' i 'receiver' do postaci domen | pozostawienie stopwords |
| --- |  --- | --- |
| final.csv | ❌ | ❌ |
| final-domain-only.csv | ✅ | ❌ |
| final-with-stop-words.csv | ❌ | ✅ |
| final-with-stop-words-domain-only.csv | ✅ | ✅ |

Wszystkie z tych zbiorów danych zostaną porównane dla każdego z modeli.

In [ ]:
final_data_frame.to_csv("dataset/processed/final-domain-only.csv", index=False)